<h1 style="font-size: 48px; color: red;">Classification à partir de la base de données My Anime List</h1>

<h1 style="font-size: 32px; color: green;">Partie I : Récupération des données à partir de l'API de MyAnimeList puis nettoyage des données</h1>

Cliquez [ici](https://myanimelist.net/apiconfig/references/api/v2) pour accéder au site de l'API

<h1 style="font-size: 22px; color: blue;">1/Récupération des données à partir de l'API de MyAnimeList</h1>


In [ ]:
import requests
import pandas as pd
import ast

#Comme l'API ne nous permet pas de récupérer les animes en groupe à travers leur ID, on récupère la liste des animés et leur caractéristiques à partir de leur le rang, dans l'ordre décroissant décroissant (en prenant les 100 1ers rangs, puis les 100 rangs suivants ainsi de suite...)

all_anime = [] 
nbr_needed = 27490  #Total number of anime on MAL as of 13/11/2024 (obtained by looking directly on the site of myanimelist)

ID = {'X-MAL-CLIENT-ID': 'c2db532c391bf31339ffd6afa650d528'} #id client obtenu après s'être inscrit sur My Anime List et avoir fait une demande
url = 'https://api.myanimelist.net/v2/anime/ranking'
parameters = {
    'ranking_type': 'all',  
    'limit': 100,  # Max limit per request, divides the total number of anime on mal
    'fields': 'id,title,mean,start_date,end_date,rank,popularity,num_list_users,num_scoring_users,nsfw,media_type,status,num_episodes,start_season,broadcast,source,average_episode_duration,rating'
}

k = 0  # offset but also the number of times the loop is used that is 27490/100 here

# Loop until we've collected the target number of anime
while k < nbr_needed:
    parameters['offset'] = k
    mal = requests.get(url, headers=ID, params=parameters)

    
    if mal.status_code == 200: # Check if the request is successful
        data = mal.json()
        all_anime.extend(data['data'])
        k += parameters['limit']

        print(str(len(all_anime)) + " collected for the moment...")
    
        if len(all_anime) >= nbr_needed:
            print("the total number of anime collected is " + str(len(all_anime)))
            break
    else :
        print("cannot retrieve more than " + str(len(all_anime))) 
        break

anime_data = pd.DataFrame(all_anime) 
print(anime_data.head(2))

print(anime_data.head())
print(anime_data.info())

#On voit que le dataframe est constitué du rang des animé et d'un "node", un dictionnaire qui contient toutes les caractéristiques de chaque anime.
#Il faut donc extraire chaque élément du dictionnaire node pour en faire des colonnes à part entière

# On extrait toutes les clés du dictionnaire 'node' et on les transforme en colonnes du dataframe
anime_data['node'] = anime_data['node'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)    
keys = set().union(*(d.keys() for d in anime_data['node'] if isinstance(d, dict)))
for y in keys:
    anime_data[f'{y}'] = anime_data['node'].apply(lambda x: x.get(y) if isinstance(x, dict) else None)

# On convertit les colonnes contenant des dictionnaires en chaînes
for column in anime_data.columns:
    if anime_data[column].map(type).eq(dict).any():
        anime_data[column] = anime_data[column].apply(lambda x: str(x) if isinstance(x, dict) else x)

print(anime_data.head())

#On supprime la colonne node qui n'apporte plus d'info
anime_data = anime_data.drop(columns=['node'])

100 collected for the moment...
200 collected for the moment...
300 collected for the moment...
400 collected for the moment...
500 collected for the moment...
600 collected for the moment...
700 collected for the moment...
800 collected for the moment...
900 collected for the moment...
1000 collected for the moment...
1100 collected for the moment...
1200 collected for the moment...
1300 collected for the moment...
1400 collected for the moment...
1500 collected for the moment...
1600 collected for the moment...
1700 collected for the moment...
1800 collected for the moment...
1900 collected for the moment...
2000 collected for the moment...
2100 collected for the moment...
2200 collected for the moment...
2300 collected for the moment...
2400 collected for the moment...
2500 collected for the moment...
2600 collected for the moment...
2700 collected for the moment...
2800 collected for the moment...
2900 collected for the moment...
3000 collected for the moment...
3100 collected for 

<h1 style="font-size: 22px; color: blue;">2/Nettoyage des données et enregistrement du dataframe en fichier csv</h1>


In [1]:
#On vérifie s'il y a des doublons
nbr_doublons = anime_data.duplicated().sum()
print(f"Il y a {nbr_doublons} doublons")

#On supprime les colonnes qui ne serviront pas pour la recommendation
anime_data=anime_data.drop(columns=['main_picture','broadcast','start_season','end_date'],axis=1)
pd.set_option('display.max_columns', None)
print(anime_data.head())

#On regarde combien de valeurs NaN il y a dans chaque colonne
for i in anime_data.columns:
    k = anime_data[i].isna().sum()
    print(f"Le nombre de NaN dans la colonne '{i}' est : {k}")

#On gère les différents types de NaN
anime_data['source'] = anime_data['source'].fillna('source_inconnue')
anime_data['rating'] = anime_data['source'].fillna('rating_inconnu')
anime_data['mean'] = anime_data['mean'].fillna(0)
anime_data = anime_data.dropna(subset=['rank'])

#On veut uniquement garder l'année dans la colonne start_date
anime_data['start_date'] = pd.to_datetime(anime_data['start_date'], errors='coerce')  
anime_data['start_year'] = anime_data['start_date'].dt.year  
anime_data=anime_data.drop(columns=['start_date'],axis=1)

anime_data = anime_data.dropna(subset=['start_year'])

#On vérifie qu'il n'y a plus de NaN
nbr_nan = anime_data.isna().sum().sum()
print(f"Il reste {nbr_nan} NaN")

#On ne garde que les colonnes numériques pour calculer la matrice de correlation
anime_data_num = anime_data.select_dtypes(include=["number"])
# Calcul de la matrice de corrélation
print(anime_data_num.corr())

#On sauvegarde le DataFrame en fichier CSV local
local_file_path = "anime_data.csv"
anime_data.to_csv(local_file_path, index=False)
print(f"Fichier CSV sauvegardé localement : {local_file_path}")


NameError: name 'anime_data' is not defined

<h1 style="font-size: 32px; color: green;">Partie II : Etudes des données et quelques visualisations</h1>


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

<h1 style="font-size: 22px; color: blue;">1/Familiarisation avec les données</h1>


In [ ]:
#On cherche à voir à quoi ressemble les données obtenues sur les animés
print(anime_data.head())
print(anime_data.describe())
print(anime_data.dtypes)

In [ ]:
#On regarde la matrice de corrélation

numerical_features = ['num_list_users', 'num_episodes', 'mean', 'rank', 'popularity', 'num_scoring_users', 'start_year']
plt.figure(figsize=(12, 8))
correlation_matrix = df[numerical_features].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matrice de correlation')
plt.show()
plt.savefig('matrice_de_co.png')

<h1 style="font-size: 22px; color: blue;">2/Etude approfondie des données</h1>

On cherche ici à tester et visualiser quelques intuitions qu'on pourrait avoir vis-à-vis des données


In [ ]:
#On va utiliser la caractéristique note moyenne des utilisateurs pour un animé dans nos études
#Or, comme une note moyenne des utilisateurs pour un anime donné était mise à 0 quand elle était inconnue (NaN), on va supprimer les animes dont la note moyenne est 0.
#De manière intuitive, il est normal de supprimer ces animes car une note moyenne par les utilisateurs de 0 est quasi impossible

print(anime_data['mean'].describe()) #On voit bien une surreprésentation des animés avec une note moyenne des utilisateurs de 0

anime_data_for_score=anime_data[anime_data['mean'] !=0] #On vient de créer un dataframe qui ne prendra pas en compte les animés avec une note moyenne des utilisateurs de 0 lorsqu'on utilisera cette même caractéristique.
print(anime_data_for_score['mean'].describe())

anime_data['start_year'].value_counts().sort_index()

In [ ]:
#1/On pense que la distribution du nombre d'animés en fonction de leur note moyenne est une courbe en cloche centré autour de 5. 
#Pour vérifier cette hypothèse, on fait un histogramme qui représente le nombre d'animes par note moyenne.
plt.figure(figsize=(10, 6))
sns.histplot(anime_data_for_score['mean'], bins=30, kde=True, color='blue')
plt.title("Distribution des animes en fonction de leurs notes moyennes")
plt.xlabel("Note moyenne")
plt.ylabel("Nbr d'animes")
plt.savefig("Nbr_animés_par_note_moyenne.png")
plt.close()

In [ ]:
#2/On pense que la production d'animés n'a cessé d'augmenter avec le temps. 
#Pour vérifier notre intuition, on représente l'évolution du nombre d'animes produits par année à l'aide d'un histogramme
plt.figure(figsize=(12, 6))
sns.countplot(x='start_year', data=anime_data, palette='viridis', hue='start_year', legend=False)
plt.xticks(rotation=90)
plt.title("Nombre d'animes produits par année")
plt.xlabel("Année de sortie")
plt.ylabel("Nbr d'animés")
plt.savefig("Nbr_animés_par_annee.png")
plt.close()

In [ ]:
#3/On veut savoir si une source est plus prolifique en animes que d'autres. 
#Pour cela, on créé un histogramme qui représente le nombre d'animes en fonction de la source dont ils sont inspirés
plt.figure(figsize=(10, 6))
source_counts = anime_data['source'].value_counts()
sns.barplot(x=source_counts.index, y=source_counts.values, palette='coolwarm', hue=source_counts.index, legend=False)
plt.xticks(rotation=90)
plt.title("Nbr d'animes par sources'")
plt.xlabel("Source")
plt.ylabel("Nombre d'animés")
plt.savefig("Nbr_animes_par_source.png")
plt.close()

In [ ]:
#4/On veut savoir si une source est plus susceptibles de produire des animés bien notés que d'autres. 
#Pour cela on fait un histogramme qui représente la note moyenne des animés d'une même source en fonction de cette source.
mean_scores_by_source = anime_data_for_score.groupby('source')['mean'].mean().sort_values(ascending=False)
plt.figure(figsize=(12, 6))
mean_scores_by_source.plot(kind='bar', color='skyblue')
plt.title("Notes moyennes par source")
plt.xlabel("Source")
plt.ylabel("Note moyenne")
plt.savefig("Notes_moyennes_par_source.png")
plt.close()

In [ ]:
#5/On veut savoir si le nombre d'utilisateurs qui a noté un anime en particulier influence la note moyenne de cette animé. 
#Pour voir cela, on fait un nuage de points dont les points sont la note moyenne d'un anime en fonction du nombre d'utilisateurs qui ont noté cette animé.
plt.figure(figsize=(10, 6))
sns.scatterplot(x='num_list_users', y='mean', data=anime_data_for_score, alpha=0.6)
plt.title("Relation entre le nombre d'utilisateurs et la note moyenne")
plt.xlabel("Nombre d'utilisateurs ayant noté")
plt.ylabel("Note moyenne")
plt.xscale('log')  # On utilise une échelle logarithmique pour mieux visualiser
plt.savefig("relation_ entre_nbr_utilisateurs_et_notes_moyennes.png")
plt.close()

In [ ]:
#6/On veut comparer le score moyen des animes regroupés par rang (par exemple : rang 1 à 100 puis rang 101 à 200, etc,...).
#Pour cela, on va créer 10 groupes de 100 animés

anime_data_for_score['Groupe'] = (anime_data_for_score['rank'] - 1) // 100 + 1 #on créé les groupes de 100 animés
anime_1000 = anime_data_for_score[anime_data_for_score['Groupe'] <= 10] #on ne garde que les 10 premiers groupes

grouped_means = anime_1000.groupby('Groupe')['mean'].mean().reset_index()
grouped_means['Groupe'] = grouped_means['Groupe'].astype(str)  # Conversion pour le graphique

plt.figure(figsize=(12, 6))
sns.barplot(x='Groupe', y='mean', data=grouped_means, palette='viridis')
plt.title("Scores moyens des animes par groupe de rangs (sans scores nuls)")
plt.xlabel("Groupe de rangs (par 100)")
plt.ylabel("Score moyen")
plt.savefig("scores_moyens_par_100_rangs.png")
plt.close()


In [ ]:
#7/On veut voir si animés sont mieux notés d'une année sur l'autre. 
#Pour cela, on fait un courbe dont chaque point est la moyenne des notes moyennes des animés d'une année en fonction de l'année
mean_score_by_year = anime_data_for_score.groupby('start_year')['mean'].mean()
plt.figure(figsize=(12, 6))
mean_score_by_year.plot(kind='line', color='green', marker='o')
plt.title("Évolution des notes moyennes des animes au fil des années")
plt.xlabel("Année")
plt.ylabel("Note moyenne")
plt.savefig("evolution_notes_moyennes_par_annee.png")
plt.close()


In [ ]:
#8/La durée d'un épisode est généralement de 20 minutes (par expérience). 
#On veut vérifier si cette intuition est vraie. Pour cela, on réalise un histogramme.

anime_data['average_episode_duration']=anime_data['average_episode_duration']/60

plt.figure(figsize=(10, 6))
sns.histplot(anime_data['average_episode_duration'].dropna(), bins=30, kde=True, color='purple')
plt.title("Nbr d'animés en fonction de la durée moyenne de leurs épisodes")
plt.xlabel("Durée moyenne des épisodes (en minutes)")
plt.ylabel("Nbr d'animés")

x_ticks = np.arange(0, anime_data['average_episode_duration'].max() + 10, 5)  # de 0 à max+10 par pas de 5 minutes
plt.xticks(x_ticks)

plt.savefig("Nbr_animés_en_fct_durée_ep.png")
plt.close()

In [ ]:
#9/On s'attend à ce que la plupart des animés ont 12 ou 24 épisodes. 
#Pour vérifier cette intuition, on fait un histogramme qui représente le nbr d'animés en fonction de leur nombres d'épisodes (si le nbr d'épisodes est de plus de 100, il ne sera pas représenté, car peu d'animés valident cette condition et ce sont des anomalies)

anime_filtered = anime_data[anime_data['num_episodes'].between(1, 100)]

episode_counts = anime_filtered['num_episodes'].value_counts().reindex(range(1, 101), fill_value=0).sort_index()

plt.figure(figsize=(18, 8))
sns.barplot(x=episode_counts.index, y=episode_counts.values, color='blue')

plt.title("Nombre d'animés en fonction du nombre d'épisodes (1 à 100)", fontsize=14)
plt.xlabel("Nombre d'épisodes", fontsize=12)
plt.ylabel("Nombre d'animés", fontsize=12)

plt.xticks(ticks=range(1, 101, 5), labels=range(1, 101, 5), rotation=45)  # Ticks tous les 5 pour la lisibilité

plt.savefig("nbr_animes_par_nombre_episodes_1_to_100.png", dpi=300)

<h1 style="font-size: 22px; color: blue;">3/Analyse de la popularité (la variable qui va être prédit à l'aide de notre classification) en fonction de diverses variables</h1>


In [ ]:
#1/Popularité en fonction des moyennes des notes
#Lors de l'analyse exploratoire (notamment avec la matrice de corrélation), nous avons constaté une corrélation significative entre la popularité et les moyennes des notes.
plt.figure(figsize=(10, 6))
plt.hexbin(x=df['mean'], y=df['popularity'], gridsize=50, cmap='Blues', mincnt=1)
plt.colorbar(label='Count')
plt.title('Popularité en fonction de la note moyenne')
plt.xlabel('Note moyenne')
plt.ylabel('Popularité')
plt.show()
plt.savefig('pop_en_fct_note_moyenne.png')

In [ ]:
#2/Popularité en fonction du nombre d'utilisateurs (qui ont regardé l'anime)
#Nous cherchons à déterminer la nature de la relation entre le nombre d'utilisateurs et la popularité des animes, car il est logique de supposer que ces deux variables sont liées.
plt.figure(figsize=(10, 6))
plt.scatter(df['num_list_users'], df['popularity'], alpha=0.6, s=10)
plt.xlim(0, df['num_list_users'].max())
plt.title('Popularité en fonction du nombre d utilisateurs')
plt.xlabel('num_list_users')
plt.ylabel('Popularité')
plt.yscale('log') #échelle logarithmique
plt.show()
plt.savefig('Popularité&nbr_users.png')

In [ ]:
#3/Popularité en fonction du type de média
plt.figure(figsize=(12, 6))
sns.boxplot(x=df['media_type'], y=df['popularity'], order=df['media_type'].value_counts().index)
plt.title('Popularité en fonction du type de media')
plt.xlabel('Media Type')
plt.ylabel('Popularité')
plt.yscale('log')
plt.xticks(rotation=45)
plt.show()
plt.savefig('Popularité&type.png')

<h1 style="font-size: 32px; color: green;">Partie III : Modélisation et prédictions à partir des classifieurs</h1>
